In [6]:
from config import *
import pandas as pd
import numpy as np
from main_package.bkt_pyKT import convert_df_strings_to_arrays
from main_package.utils import data_path_to_abs_path
from main_package.bkt_pyKT_per_skill import train_bkt, evaluate_bkt
import copy

In [2]:
df_train = pd.read_csv(data_path_to_abs_path('isaac/pyKT_processed/train_valid_sequences.csv'))
convert_df_strings_to_arrays(df_train)
df_train.head()

,fold,uid,questions,concepts,responses,selectmasks,is_repeat
0,0,23131,"[389, 389, 389, 394, 394, 394, 3, 3, 3, 182, 7...","[7, 2, 66, 2, 39, 66, 7, 8, 5, 31, 31, 43, 8, ...","[1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, ..."
1,0,21411,"[395, 104, 104, 355, 355, 178, -1, -1, -1, -1,...","[52, 7, 52, 52, 12, 52, -1, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1,...","[1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1,...","[0, 0, 1, 0, 1, 0, -1, -1, -1, -1, -1, -1, -1,..."
2,0,10046,"[401, 401, 401, 401, 401, 401, 403, 403, 403, ...","[32, 3, 33, 79, 10, 9, 7, 8, 11, 12, 3, 4, 39,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ..."
3,0,12053,"[213, 213, 214, 214, 130, 130, 348, 399, 193, ...","[13, 30, 13, 30, 13, 30, 83, 69, 48, -1, -1, -...","[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1...","[0, 1, 0, 1, 0, 1, 0, 0, 0, -1, -1, -1, -1, -1..."
4,0,8962,"[87, 425, 66, 66, -1, -1, -1, -1, -1, -1, -1, ...","[46, 0, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[1, 1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -...","[1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0, 0, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [10]:
df_sample = copy.deepcopy(df_train[:500])
for column_name in ['questions', 'concepts', 'responses', 'is_repeat', 'selectmasks']:
    df_sample[column_name] = df_sample.apply(
        lambda row: row[column_name][row['selectmasks']=='1'],
        axis=1,
    )
df_sample = df_sample.drop(columns='selectmasks')
df_sample.head()

,fold,uid,questions,concepts,responses,is_repeat
0,0,23131,"[389, 389, 389, 394, 394, 394, 3, 3, 3, 182, 7...","[7, 2, 66, 2, 39, 66, 7, 8, 5, 31, 31, 43, 8, ...","[1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, ..."
1,0,21411,"[395, 104, 104, 355, 355, 178]","[52, 7, 52, 52, 12, 52]","[0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0]"
2,0,10046,"[401, 401, 401, 401, 401, 401, 403, 403, 403, ...","[32, 3, 33, 79, 10, 9, 7, 8, 11, 12, 3, 4, 39,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ..."
3,0,12053,"[213, 213, 214, 214, 130, 130, 348, 399, 193]","[13, 30, 13, 30, 13, 30, 83, 69, 48]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 1, 0, 1, 0, 1, 0, 0, 0]"
4,0,8962,"[87, 425, 66, 66]","[46, 0, 0, 1]","[1, 1, 0, 0]","[0, 0, 0, 1]"
